# MHW 3

> Выполнил: Разуваев Никита Сергеевич

## Задание 1

* Из всех имеющихся данных необходимо составить датасет из минимум **8 логических признаков**. Под логическим признаком имеется в виду то, что например кодирование категориального признака или текста в виде OheHot\BoW не считается за кучу признаков - конкретный текст считается за 1 признак, категориальный признак считается за 1 признак.

* Всю работу по обработке данных и сборке финального датасета необходимо провести с помощью спарка. Финальный датасет должен быть закодирован в формате Vowpal Wabbit.

* Важно Следите за тем, чтобы исходное количество объектов не изменилось. При неаккуратной работе с join в sql вы можете начать терять какие-то записи, что плохо. Как минимум это означает, что вы не сможете собрать такие же признаки для тестовой выборки, которую предлагают в задаче.

* Важно Обратите внимание, что в задаче стоит вопрос именно про контекстуальную рекламу, а не про всю существующую.

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql import functions as F

import os
import glob


sc = pyspark.SparkContext(appName="lsml-app-mhw3")
se = SparkSession(sc)

In [2]:
%cd "~/storage/avito-context-ad-clicks"
! ls

/home/ubuntu/storage/avito-context-ad-clicks
AdsInfo.tsv   PhoneRequestsStream.tsv  trainSearchStream.tsv
Category.tsv  SearchInfo.tsv	       UserInfo.tsv
Location.tsv  submissions	       VisitsStreamNoHeader.tsv
other	      testSearchStream.tsv     VisitsStream.tsv


### Перекинем данные в parquet

In [3]:
! hdfs dfs -ls /user/avito/data

Found 9 items
-rw-r--r--   1 ubuntu hadoop  5350670676 2022-02-07 08:17 /user/avito/data/AdsInfo.tsv
-rw-r--r--   1 ubuntu hadoop         752 2022-03-15 13:19 /user/avito/data/Category.tsv
-rw-r--r--   1 ubuntu hadoop       58976 2022-03-15 13:19 /user/avito/data/Location.tsv
-rw-r--r--   1 ubuntu hadoop   630319200 2022-03-15 13:19 /user/avito/data/PhoneRequestsStream.tsv
-rw-r--r--   1 ubuntu hadoop  9469373867 2022-02-07 05:10 /user/avito/data/SearchInfo.tsv
-rw-r--r--   1 ubuntu hadoop   104614699 2022-03-15 13:19 /user/avito/data/UserInfo.tsv
-rw-r--r--   1 ubuntu hadoop 13180996392 2022-03-15 13:29 /user/avito/data/VisitsStream.tsv
-rw-r--r--   1 ubuntu hadoop   557829937 2022-03-15 13:19 /user/avito/data/testSearchStream.tsv
-rw-r--r--   1 ubuntu hadoop 11023566785 2022-02-07 05:16 /user/avito/data/trainSearchStream.tsv


In [ ]:
# ! sudo -u hdfs hdfs balancer 
# ! hdfs dfs -rm -r /user/avito/parquet
# ! hdfs dfs -mkdir -p /user/avito/parquet

In [4]:
data_list = glob.glob(os.path.join('*.tsv'))
data_list.remove('VisitsStreamNoHeader.tsv')
data_list

['SearchInfo.tsv',
 'PhoneRequestsStream.tsv',
 'AdsInfo.tsv',
 'trainSearchStream.tsv',
 'Category.tsv',
 'Location.tsv',
 'VisitsStream.tsv',
 'UserInfo.tsv',
 'testSearchStream.tsv']

In [38]:
data_tsv = {
    fp.split('.')[0]: se.read.option("mode", "DROPMALFORMED")
    .option('sep', "\t")
    .csv(os.path.join('/user/avito/data', fp), header=True, inferSchema=True)
    for fp in data_list
}

In [41]:
for tbl in data_tsv:
    print(f'Saving {tbl} to parquet...')
    data_tsv[tbl].write.parquet(f"/user/avito/parquet/{tbl}.parquet")

Saving SearchInfo to parquet...
Saving PhoneRequestsStream to parquet...
Saving AdsInfo to parquet...
Saving trainSearchStream to parquet...
Saving Category to parquet...
Saving Location to parquet...
Saving VisitsStream to parquet...
Saving UserInfo to parquet...
Saving testSearchStream to parquet...


In [46]:
! hdfs dfs -ls /user/avito/parquet

Found 9 items
drwxr-xr-x   - ubuntu hadoop          0 2022-03-15 13:54 /user/avito/parquet/AdsInfo.parquet
drwxr-xr-x   - ubuntu hadoop          0 2022-03-15 13:58 /user/avito/parquet/Category.parquet
drwxr-xr-x   - ubuntu hadoop          0 2022-03-15 13:58 /user/avito/parquet/Location.parquet
drwxr-xr-x   - ubuntu hadoop          0 2022-03-15 13:52 /user/avito/parquet/PhoneRequestsStream.parquet
drwxr-xr-x   - ubuntu hadoop          0 2022-03-15 13:52 /user/avito/parquet/SearchInfo.parquet
drwxr-xr-x   - ubuntu hadoop          0 2022-03-15 14:02 /user/avito/parquet/UserInfo.parquet
drwxr-xr-x   - ubuntu hadoop          0 2022-03-15 14:02 /user/avito/parquet/VisitsStream.parquet
drwxr-xr-x   - ubuntu hadoop          0 2022-03-15 14:02 /user/avito/parquet/testSearchStream.parquet
drwxr-xr-x   - ubuntu hadoop          0 2022-03-15 13:58 /user/avito/parquet/trainSearchStream.parquet


### Создание логических признаков

In [5]:
data = {
    fp.split('.')[0]:
    se.read.parquet('/user/avito/parquet/{}.parquet'.format(fp.split('.')[0]))
    for fp in data_list
}
for tbl in data:
    data[tbl].registerTempTable(tbl)
data

{'SearchInfo': DataFrame[SearchID: int, SearchDate: string, IPID: int, UserID: int, IsUserLoggedOn: int, SearchQuery: string, LocationID: int, CategoryID: int, SearchParams: string],
 'PhoneRequestsStream': DataFrame[UserID: int, IPID: int, AdID: int, PhoneRequestDate: string],
 'AdsInfo': DataFrame[AdID: int, LocationID: int, CategoryID: int, Params: string, Price: double, Title: string, IsContext: int],
 'trainSearchStream': DataFrame[SearchID: int, AdID: int, Position: int, ObjectType: int, HistCTR: double, IsClick: int],
 'Category': DataFrame[CategoryID: int, Level: int, ParentCategoryID: int, SubcategoryID: int],
 'Location': DataFrame[LocationID: int, Level: int, RegionID: int, CityID: int],
 'VisitsStream': DataFrame[UserID: int, IPID: int, AdID: int, ViewDate: string],
 'UserInfo': DataFrame[UserID: int, UserAgentID: int, UserAgentOSID: int, UserDeviceID: int, UserAgentFamilyID: int],
 'testSearchStream': DataFrame[ID: int, SearchID: int, AdID: int, Position: int, ObjectType: 

In [6]:
import string
import re


def to_number(raw_value):
    try:
        return float(raw_value)
    except:
        return 0.0

se.udf.register("to_number", to_number, "float")


def slugify(text):
    if not text:
        return ""
    punct = r'[-!\"#$%&\(\)\*\+,-\.\/:;<=>\?@\[\\\]\^_`{|}~\s]+'
    text = re.sub(punct, ' ', text)
    text = re.sub(r'[^a-zA-Zа-яА-Я0-9\s]', '', text.lower())
    text = re.sub(r'[0-9]+', '_num', text.lower())
    return text

se.udf.register("slugify", slugify, "string")


def extract_categories(text):
    """
    Вытаскиваем номер категории рег. выражением
    """
    if not text:
        return ""
    regexp = r'(\d+):'
    return ' '.join(re.findall(regexp, text))

se.udf.register("extract_categories", extract_categories, "string")


def extract_text_from_categories(text):
    """
    Вытаскиваем текст категории рег. выражением
    """
    if not text:
        return ""
    punct = r'[-!\"#$%&\(\)\*\+,-\.\/:;<=>\?@\[\\\]\^_`{|}~\s]+'
    text = re.sub(punct, ' ', text)
    regexp = r'\'([a-zA-Zа-яА-Я0-9\s]+)\''
    return ' '.join(re.findall(regexp, text.lower()))

se.udf.register("extract_text_from_categories", extract_text_from_categories, "string")

<function __main__.extract_text_from_categories(text)>

In [7]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
_ = '{132:\'Видео, DVD и Blu-ray плееры\'}'
print(extract_categories(_))
print(extract_text_from_categories(_))

132
видео dvd и blu ray плееры


#### Сырые таблички

In [9]:
se.sql("""
    SELECT * 
    FROM trainSearchStream
    LIMIT 10
""").toPandas()

,SearchID,AdID,Position,ObjectType,HistCTR,IsClick
0,2,11441863,1,3,0.001804,0.0
1,2,22968355,7,3,0.004723,0.0
2,3,212187,7,3,0.029701,0.0
3,3,34084553,1,3,0.004300,0.0
4,3,36256251,2,2,NaN,NaN
5,4,2073399,6,1,NaN,NaN
6,4,6046052,7,1,NaN,NaN
7,4,17544913,8,1,NaN,NaN
8,4,20653823,1,3,0.003049,0.0
9,4,24129570,2,2,NaN,NaN


In [10]:
se.sql("""
    SELECT * 
    FROM testSearchStream
    LIMIT 10
""").toPandas()

,ID,SearchID,AdID,Position,ObjectType,HistCTR
0,1,1,10915336,1,3,0.004999
1,2,1,12258424,6,1,NaN
2,3,1,15952399,2,1,NaN
3,4,1,17092357,8,1,NaN
4,5,1,31173651,7,3,0.045094
5,6,7,9041972,1,3,0.007032
6,7,28,14309446,7,3,0.016943
7,8,28,15753167,1,3,0.054339
8,9,28,20744962,2,2,NaN
9,10,44,2157040,6,1,NaN


In [11]:
se.sql("""
    SELECT * 
    FROM SearchInfo
    LIMIT 10
""").toPandas()

,SearchID,SearchDate,IPID,UserID,IsUserLoggedOn,SearchQuery,LocationID,CategoryID,SearchParams
0,1,2015-05-18 19:54:32.0,1717090,3640266,0,None,1729,5,None
1,2,2015-05-12 14:21:28.0,1731568,769304,0,None,697,50,None
2,3,2015-05-12 07:09:42.0,793143,640089,0,None,1261,12,None
3,4,2015-05-10 18:11:01.0,898705,3573776,0,None,3960,22,"{83:'Обувь', 175:'Женская одежда', 88:'38'}"
4,5,2015-04-25 13:04:09.0,2009707,320674,0,None,547,1,None
5,6,2015-05-07 16:49:15.0,1658456,1665156,0,None,926,27,None
6,7,2015-05-14 23:07:27.0,1849117,3434614,0,None,44,500001,None
7,8,2015-05-09 09:10:06.0,572585,905821,0,None,2694,12,None
8,9,2015-05-02 20:14:15.0,77162,1106541,0,None,576,4,None
9,10,2015-05-15 17:38:46.0,1371072,3310798,0,None,1110,12,None


In [12]:
se.sql("""
    SELECT * 
    FROM AdsInfo
    LIMIT 10
""").toPandas()

,AdID,LocationID,CategoryID,Params,Price,Title,IsContext
0,1,343,43,"{1283:'С пробегом', 633:'Синий', 1159:0, 210:'...",160000.0,"Toyota Estima, 1993",0
1,2,992,34,"{817:'Кузов', 5:'Запчасти', 598:'Для автомобил...",750.0,Передние брызговики Форд Фокус 2 родные,0
2,3,3771,53,{181:'Промышленное'},18000.0,Дровокол,0
3,4,4294,57,{130:'Приборы и аксессуары'},1500.0,Продам ходули складные,0
4,5,1344,34,"{817:'Автосвет', 5:'Запчасти', 598:'Для автомо...",800.0,Поворотник R - Carina (20317),0
5,6,3953,34,"{817:'Кузов', 5:'Запчасти', 598:'Для автомобил...",0.0,ВАЗ дверь 2106 передняя левая,0
6,7,1330,26,"{132:'Видео, DVD и Blu-ray плееры'}",30.0,Видео,0
7,8,3022,22,"{83:'Свадебные платья', 476:'42–44 (S)', 175:'...",7000.0,Дизайнерское платье,0
8,9,2840,47,{127:'Игрушки'},2460.0,Светодиодный конструктор lite brix Подиум,0
9,10,2277,54,"{549:'3', 201:'Продам', 498:'Кирпичный', 499:'...",7000000.0,"3-к квартира, 92 м², 4/4 эт.",0


In [13]:
se.sql("""
    SELECT * 
    FROM PhoneRequestsStream
    LIMIT 10
""").toPandas()

,UserID,IPID,AdID,PhoneRequestDate
0,730655,381437,12733961,2015-05-03 22:05:28.0
1,1277241,189160,21293894,2015-05-03 22:05:28.0
2,1557310,1893598,10107772,2015-05-03 22:05:28.0
3,1680741,460254,2689163,2015-05-03 22:05:28.0
4,1971684,525104,149032,2015-05-03 22:05:28.0
5,2561330,1914356,30452389,2015-05-03 22:05:28.0
6,4177000,1276038,8359959,2015-05-03 22:05:28.0
7,4263912,274146,32468754,2015-05-03 22:05:28.0
8,2400896,89495,21456480,2015-05-03 22:05:29.0
9,2472835,91497,5727048,2015-05-03 22:05:29.0


In [14]:
se.sql("""
    SELECT * 
    FROM Category
    LIMIT 10
""").toPandas()

,CategoryID,Level,ParentCategoryID,SubcategoryID
0,0,1,10,45
1,1,2,9,45
2,2,3,12,5
3,3,3,9,25
4,4,3,2,39
5,5,3,2,35
6,6,3,4,53
7,7,2,2,45
8,8,3,2,31
9,9,3,9,18


In [15]:
se.sql("""
    SELECT * 
    FROM Location
    LIMIT 10
""").toPandas()

,LocationID,Level,RegionID,CityID
0,7,3,83,2386
1,23,3,28,3224
2,26,3,41,1316
3,30,3,63,2565
4,32,3,28,2819
5,35,3,38,1564
6,39,3,55,1881
7,44,3,55,746
8,53,3,83,3588
9,63,3,28,2026


In [16]:
se.sql("""
    SELECT * 
    FROM VisitsStream
    LIMIT 10
""").toPandas()

,UserID,IPID,AdID,ViewDate
0,3339459,2141522,17013380,2015-05-19 17:47:02.0
1,3472056,468478,9250676,2015-05-19 17:47:02.0
2,3498500,2194994,16265489,2015-05-19 17:47:02.0
3,3581440,167293,29431869,2015-05-19 17:47:02.0
4,3756375,1294847,311552,2015-05-19 17:47:02.0
5,3891751,1190064,9709139,2015-05-19 17:47:02.0
6,3971739,1811955,16457203,2015-05-19 17:47:02.0
7,4006974,1796040,36742059,2015-05-19 17:47:02.0
8,4102599,678975,32139132,2015-05-19 17:47:02.0
9,4106979,1479340,32656590,2015-05-19 17:47:02.0


In [17]:
se.sql("""
    SELECT * 
    FROM UserInfo
    LIMIT 10
""").toPandas()

,UserID,UserAgentID,UserAgentOSID,UserDeviceID,UserAgentFamilyID
0,1,44073,30,2019,9
1,2,12505,20,2014,85
2,3,24256,20,2014,64
3,4,57133,20,2014,25
4,5,57133,20,2014,25
5,6,57201,20,2014,85
6,7,22293,20,2014,25
7,8,19983,35,3870,63
8,9,12505,20,2014,85
9,10,60018,43,2014,85


#### Обработка таблиц

In [9]:
# Признаки из дат
# Текст запроса
# Параметры поиска
se.sql("""
    SELECT SearchID, UserID, IsUserLoggedOn, LocationID, CategoryID,
        dayofweek(SearchDate) as DayOfWeek, dayofmonth(SearchDate) as DayOfMonth, month(SearchDate) as Month,
        slugify(lower(SearchQuery)) as SearchQuery,
        extract_categories(SearchParams) as SearchParams_n, extract_text_from_categories(SearchParams) as SearchParams_t
    FROM SearchInfo
""").registerTempTable("SearchInfo_f1")

In [10]:
se.sql("""
    SELECT *
    FROM SearchInfo_f1
    LIMIT 10
""").toPandas()

,SearchID,UserID,IsUserLoggedOn,LocationID,CategoryID,DayOfWeek,DayOfMonth,Month,SearchQuery,SearchParams_n,SearchParams_t
0,1,3640266,0,1729,5,2,18,5,,,
1,2,769304,0,697,50,3,12,5,,,
2,3,640089,0,1261,12,3,12,5,,,
3,4,3573776,0,3960,22,1,10,5,,83 175 88,обувь женская одежда 38
4,5,320674,0,547,1,7,25,4,,,
5,6,1665156,0,926,27,5,7,5,,,
6,7,3434614,0,44,500001,5,14,5,,,
7,8,905821,0,2694,12,7,9,5,,,
8,9,1106541,0,576,4,7,2,5,,,
9,10,3310798,0,1110,12,6,15,5,,,


In [11]:
se.sql("""
    SELECT AdID, LocationID, CategoryID, IsContext,
        extract_categories(Params) as Params_n, extract_text_from_categories(Params) as Params_t,
        int(to_number(Price)) as Price, 
        slugify(lower(Title)) as Title
    FROM AdsInfo
""").registerTempTable("AdsInfo_f1")

In [22]:
se.sql("""
    SELECT * 
    FROM AdsInfo_f1
    LIMIT 10
""").toPandas()

,AdID,LocationID,CategoryID,IsContext,Params_n,Params_t,Price,Title
0,1,343,43,0,1283 633 1159 210 184 1162 1165 1135 1329 1138...,с пробегом синий toyota 0 4 999 2 0 estima пол...,160000,toyota estima _num
1,2,992,34,0,817 5 598,кузов запчасти для автомобилей,750,передние брызговики форд фокус _num родные
2,3,3771,53,0,181,промышленное,18000,дровокол
3,4,4294,57,0,130,приборы и аксессуары,1500,продам ходули складные
4,5,1344,34,0,817 5 598,автосвет запчасти для автомобилей,800,поворотник r carina _num
5,6,3953,34,0,817 5 598,кузов запчасти для автомобилей,0,ваз дверь _num передняя левая
6,7,1330,26,0,132,видео dvd и blu ray плееры,30,видео
7,8,3022,22,0,83 476 175,свадебные платья 175,7000,дизайнерское платье
8,9,2840,47,0,127,игрушки,2460,светодиодный конструктор lite brix подиум
9,10,2277,54,0,549 201 498 499 566 575,3 продам кирпичный вторичка не первый 5,7000000,_num к квартира _num м _num _num эт


#### Собираем все вместе

* **Train**

In [12]:
se.sql("""
SELECT 
    a.SearchID, a.AdID, a.Position, a.ObjectType, to_number(a.HistCTR) as HistCTR, int(to_number(a.IsClick)) as IsClick,
    si.UserID, si.IsUserLoggedOn, si.LocationID, si.CategoryID, si.DayOfWeek, si.DayOfMonth, si.Month,
    si.SearchQuery, si.SearchParams_n, si.SearchParams_t,
    ai.LocationID as AdLocationID, ai.CategoryID as AdCategoryID, ai.IsContext, ai.Params_n, ai.Params_t, ai.Price, ai.Title
FROM 
    trainSearchStream a 
    LEFT JOIN SearchInfo_f1 si ON a.SearchID = si.SearchID
    LEFT JOIN AdsInfo_f1 ai ON a.AdID = ai.AdID
""").registerTempTable("trainSearchStream_f1")

In [13]:
se.sql("""
SELECT 
    a.SearchID, a.AdID, a.Position, a.ObjectType, a.HistCTR, a.IsClick,
    a.UserID, a.IsUserLoggedOn, a.LocationID, a.CategoryID, a.DayOfWeek, a.DayOfMonth, a.Month,
    a.SearchQuery, a.SearchParams_n, a.SearchParams_t,
    a.AdLocationID, a.AdCategoryID, a.IsContext, a.Params_n, a.Params_t, a.Price, a.Title,
    ui.UserAgentID, ui.UserAgentOSID, ui.UserDeviceID, ui.UserAgentFamilyID,
    lc.Level as LocLevel, lc.RegionID as LocRegionID, lc.CityID as LocRegionID,
    adlc.Level as AdLocLevel, adlc.RegionID as AdLocRegionID, adlc.CityID as AdLocCityID
FROM 
    trainSearchStream_f1 a 
    LEFT JOIN UserInfo ui ON a.UserID = ui.UserID
    LEFT JOIN Location lc ON a.LocationID = lc.LocationID
    LEFT JOIN Location adlc ON a.AdLocationID = adlc.LocationID
""").fillna(-1).registerTempTable("trainSearchStream_f2")

In [14]:
train_data, val_data = se.sql("""
SELECT *
FROM trainSearchStream_f2
""").randomSplit([0.8, 0.2], 15032022)

In [ ]:
train = train_data.rdd.cache()
val = val_data.rdd.cache()

In [43]:
def convert_to_vw(data):
    target = data['IsClick']
    if target == 0:
        target = -1
    
    
    template = """{target} |numeric histctr: {HistCTR} isuserloggedon:{IsUserLoggedOn} iscontext:{IsContext} price:{Price}
|a {Position} |b {objectType}
|c {UserID} |d {LocationID} |e {CategoryID} |f {DayOfWeek} |g {DayOfMonth} |h {Month}
|i {SearchQuery} |j {SearchParams_n} |k {SearchParams_t} |l {AdLocationID} |m {AdCategoryID}
|n {Params_n} |o {Params_t} |p {Title} |q {UserAgentID} |r {UserAgentOSID} |s {UserDeviceID}
|t {UserAgentFamilyID} |u {LocLevel} |v {LocRegionID} |w {AdLocLevel} |x {AdLocRegionID}
|y {AdLocCityID}""".replace('\n', ' ')
    return template.format(
        target=target,
        Position=data['Position'],
        objectType=data['ObjectType'],
        HistCTR=data['HistCTR'],
        IsUserLoggedOn=data['IsUserLoggedOn'],
        UserID=data['UserID'],
        IsContext=data['IsContext'],
        Price=data['Price'],
        LocationID=data['LocationID'],
        CategoryID=data['CategoryID'],
        DayOfWeek=data['DayOfWeek'],
        Month=data['Month'],
        DayOfMonth=data['DayOfMonth'],
        SearchQuery=data['SearchQuery'],
        SearchParams_n=data['SearchParams_n'],
        SearchParams_t=data['SearchParams_t'],
        AdLocationID=data['AdLocationID'],
        AdCategoryID=data['AdCategoryID'],
        Params_n=data['Params_n'],
        Params_t=data['Params_t'],
        Title=data['Title'],
        UserAgentID=data['UserAgentID'],
        UserAgentOSID=data['UserAgentOSID'],
        UserDeviceID=data['UserDeviceID'],
        UserAgentFamilyID=data['UserAgentFamilyID'],
        LocLevel=data['LocLevel'],
        LocRegionID=data['LocRegionID'],
        AdLocLevel=data['AdLocLevel'],
        AdLocRegionID=data['AdLocRegionID'],
        AdLocCityID=data['AdLocCityID'],
    )

In [ ]:
! hdfs dfs -rm -r '/user/avito/vw/train.vw'
train.map(lambda x: convert_to_vw(x)).coalesce(1).saveAsTextFile('/user/avito/vw/train.vw')

Deleted /user/avito/vw/train.vw


In [34]:
train.take(2)[1]

Row(SearchID=208, AdID=8169231, Position=8, ObjectType=1, HistCTR=0.0, IsClick=0, UserID=3497664, IsUserLoggedOn=0, LocationID=3960, CategoryID=35, DayOfWeek=1, DayOfMonth=10, Month=5, SearchQuery='', SearchParams_n='', SearchParams_t='', AdLocationID=3960, AdCategoryID=35, IsContext=0, Params_n='', Params_t='', Price=2300, Title='подставка под цветы кашпо на _num горшка', UserAgentID=22293, UserAgentOSID=20, UserDeviceID=2014, UserAgentFamilyID=25, LocLevel=2, LocRegionID=18, LocRegionID=623, AdLocLevel=2, AdLocRegionID=18, AdLocCityID=623)

In [ ]:
! hdfs dfs -rm -r '/user/avito/vw/val.vw'
val.map(lambda x: convert_to_vw(x)).coalesce(1).saveAsTextFile('/user/avito/vw/val.vw')

* **Test**

In [38]:
se.sql("""
SELECT ID, SearchID, AdID, Position, ObjectType, to_number(HistCTR) as HistCTR
FROM testSearchStream
LIMIT 10
""").toPandas()

,ID,SearchID,AdID,Position,ObjectType,HistCTR
0,1,1,10915336,1,3,0.004999
1,2,1,12258424,6,1,0.000000
2,3,1,15952399,2,1,0.000000
3,4,1,17092357,8,1,0.000000
4,5,1,31173651,7,3,0.045094
5,6,7,9041972,1,3,0.007032
6,7,28,14309446,7,3,0.016943
7,8,28,15753167,1,3,0.054339
8,9,28,20744962,2,2,0.000000
9,10,44,2157040,6,1,0.000000


In [ ]:
se.sql("""
SELECT 
    a.ID, a.SearchID, a.AdID, a.Position, a.ObjectType, to_number(a.HistCTR) as HistCTR,
    si.UserID, si.IsUserLoggedOn, si.LocationID, si.CategoryID, si.DayOfWeek, si.DayOfMonth, si.Month,
    si.SearchQuery, si.SearchParams_n, si.SearchParams_t,
    ai.LocationID as AdLocationID, ai.CategoryID as AdCategoryID, ai.IsContext, ai.Params_n, ai.Params_t, ai.Price, ai.Title
FROM 
    testSearchStream a 
    LEFT JOIN SearchInfo_f1 si ON a.SearchID = si.SearchID
    LEFT JOIN AdsInfo_f1 ai ON a.AdID = ai.AdID
""").registerTempTable("testSearchStream_f1")

In [ ]:
se.sql("""
SELECT 
    a.ID, a.SearchID, a.AdID, a.Position, a.ObjectType, a.HistCTR,
    a.UserID, a.IsUserLoggedOn, a.LocationID, a.CategoryID, a.DayOfWeek, a.DayOfMonth, a.Month,
    a.SearchQuery, a.SearchParams_n, a.SearchParams_t,
    a.AdLocationID, a.AdCategoryID, a.IsContext, a.Params_n, a.Params_t, a.Price, a.Title,
    ui.UserAgentID, ui.UserAgentOSID, ui.UserDeviceID, ui.UserAgentFamilyID,
    lc.Level as LocLevel, lc.RegionID as LocRegionID, lc.CityID as LocRegionID,
    adlc.Level as AdLocLevel, adlc.RegionID as AdLocRegionID, adlc.CityID as AdLocCityID
FROM 
    testSearchStream_f1 a 
    LEFT JOIN UserInfo ui ON a.UserID = ui.UserID
    LEFT JOIN Location lc ON a.LocationID = lc.LocationID
    LEFT JOIN Location adlc ON a.AdLocationID = adlc.LocationID
""").fillna(-1).registerTempTable("testSearchStream_f2")

In [ ]:
test_data = se.sql("""
SELECT *
FROM testSearchStream_f2
""")

In [ ]:
test = test_data.rdd.cache()

In [ ]:
def test_convert_to_vw(data):
    
    template = """|numeric histctr: {HistCTR} isuserloggedon:{IsUserLoggedOn} iscontext:{IsContext} price:{Price}
|a {Position} |b {objectType}
|c {UserID} |d {LocationID} |e {CategoryID} |f {DayOfWeek} |g {DayOfMonth} |h {Month}
|i {SearchQuery} |j {SearchParams_n} |k {SearchParams_t} |l {AdLocationID} |m {AdCategoryID}
|n {Params_n} |o {Params_t} |p {Title} |q {UserAgentID} |r {UserAgentOSID} |s {UserDeviceID}
|t {UserAgentFamilyID} |u {LocLevel} |v {LocRegionID} |w {AdLocLevel} |x {AdLocRegionID}
|y {AdLocCityID}""".replace('\n', ' ')
    return template.format(
        Position=data['Position'],
        objectType=data['ObjectType'],
        HistCTR=data['HistCTR'],
        IsUserLoggedOn=data['IsUserLoggedOn'],
        UserID=data['UserID'],
        IsContext=data['IsContext'],
        Price=data['Price'],
        LocationID=data['LocationID'],
        CategoryID=data['CategoryID'],
        DayOfWeek=data['DayOfWeek'],
        Month=data['Month'],
        DayOfMonth=data['DayOfMonth'],
        SearchQuery=data['SearchQuery'],
        SearchParams_n=data['SearchParams_n'],
        SearchParams_t=data['SearchParams_t'],
        AdLocationID=data['AdLocationID'],
        AdCategoryID=data['AdCategoryID'],
        Params_n=data['Params_n'],
        Params_t=data['Params_t'],
        Title=data['Title'],
        UserAgentID=data['UserAgentID'],
        UserAgentOSID=data['UserAgentOSID'],
        UserDeviceID=data['UserDeviceID'],
        UserAgentFamilyID=data['UserAgentFamilyID'],
        LocLevel=data['LocLevel'],
        LocRegionID=data['LocRegionID'],
        AdLocLevel=data['AdLocLevel'],
        AdLocRegionID=data['AdLocRegionID'],
        AdLocCityID=data['AdLocCityID'],
    )

In [ ]:
! hdfs dfs -rm -r '/user/avito/vw/test.vw'
test.map(lambda x: convert_to_vw(x)).coalesce(1).saveAsTextFile('/user/avito/vw/test.vw')

## Задание 2

* Разделите ваш датасет на обучающую и тестовую выборку. Обучите логистическую регрессию с помощью Vowpal Wabbit.

* Сделайте бинарные предсказания и посчитайте Accuracy, Presicion и Recall на тестовой выборке. Сделайте предсказания вероятностей и посчитайте LogLoss. (Эта же метрика используется в Kaggle для оценивания вашего решения).

* Посчитайте ответы для тестовой выборки, которая предлагается в задании и отправьте в Kaggle (само соревнование уже давно закончилось, но вы все еще можете присылать туда свои решения и система будет вас оценивать). В решении приложите скриншот того, какой скор вам выдал Kaggle.

In [ ]:
! sudo wget http://finance.yendor.com/ML/VW/Binaries/vw-8.20190624 -O /usr/bin/vw
! sudo chmod +x /usr/bin/vw
! sudo chown ubuntu /usr/bin/vw

In [ ]:
! sudo apt-get update -y && sudo apt-get install graphviz -y

In [ ]:
! pip install numpy pandas sklearn dateparser pandarallel ipywidgets catboost graphviz

In [ ]:
! /opt/conda/bin/jupyter nbextension enable --py widgetsnbextension

In [ ]:
# Проверяем, что vw работает
! vw --help | head

In [ ]:
! hdfs dfs -get /user/avito/vw/train.vw "~/storage/avito-context-ad-clicks/vw/train.vw"

## Задание 3

Несмотря на то, что мы просим вас считать качество (и на тестовой и в Kaggle), оно никак не будет влиять на основные баллы. Основная цель этого задания состоит в том, чтобы вы потренировались в End-to-end работе с ML задачей на больших данных - от сбора датасета до получения модели и ее оценки.

Однако, топ 5 студентов (в каждой группе отдельно) с самым больших качеством (по Kaggle) получат до 5 дополнительных баллов в зависимости от полученой метрики качества (за самое лучшее качество +5, за второе место +4 и так далее).

Важно В сдаваемом ноубуке должны присутствовать такие элементы

* Код на Spark, который собирает датасет
* Команды запуска VW для его обучения
* Подсчитанные метрики качества (и соответственно код, который их считает)
* Скриншот из Kaggle, на котором видно вашу посылку и подсчитанную метрику качества